In [266]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import set_config
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
import seaborn as sns
import numpy as np
import warnings

In [267]:
warnings.filterwarnings('ignore')
set_config(transform_output="pandas")

In [268]:
target_column = "num_sold"

In [269]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 150)

In [270]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train['dataset'] = True
test['dataset'] = False

data = pd.concat([train, test]).reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328680 entries, 0 to 328679
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        328680 non-null  int64  
 1   date      328680 non-null  object 
 2   country   328680 non-null  object 
 3   store     328680 non-null  object 
 4   product   328680 non-null  object 
 5   num_sold  221259 non-null  float64
 6   dataset   328680 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 15.4+ MB


In [271]:
data['date'] = pd.to_datetime(data['date'])

In [272]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328680 entries, 0 to 328679
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        328680 non-null  int64         
 1   date      328680 non-null  datetime64[ns]
 2   country   328680 non-null  object        
 3   store     328680 non-null  object        
 4   product   328680 non-null  object        
 5   num_sold  221259 non-null  float64       
 6   dataset   328680 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 15.4+ MB


In [273]:
def shift_target_column_to_end(data):
    target_data = data.pop(target_column)
    data[target_column] = target_data
    return data

In [ ]:
def date_features(data):
    data['date_month'] = data['date'].dt.month
    data['date_year'] = data['date'].dt.year
    data['date_day'] = data['date'].dt.day
    data['date_day_of_week'] = data['date'].dt.day_of_week
    data['date_day_of_year'] = data['date'].dt.day_of_year
    data['date_quarter'] = data['date'].dt.quarter

    data['date_month_start'] = data['date'].dt.is_month_start
    data['date_month_end'] = data['date'].dt.is_month_end

    data['date_year_start'] = data['date'].dt.is_year_start
    data['date_year_end'] = data['date'].dt.is_year_end

    data['date_quarter_start'] = data['date'].dt.is_quarter_start
    data['date_quarter_end'] = data['date'].dt.is_quarter_end

    data['date_weekend'] = (data['date_day_of_week'] >= 5)

    data["days_since_start"] = (data["date"] - data.iloc[0].date).dt.days

    data = shift_target_column_to_end(data)

    data['te_exp_mean'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.expanding().mean().shift()).fillna(0)
    data['te_exp_median'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.expanding().median().shift()).fillna(0)
    data['te_exp_std'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.expanding().std().shift()).fillna(1)
    data['te_exp_var'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.expanding().var().shift()).fillna(1)
    data['te_exp_min'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.expanding().min().shift()).fillna(0)
    data['te_exp_max'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.expanding().max().shift()).fillna(0)

    data['te_lag_1_day'] = data.groupby(by=['country','store','product'])[target_column].shift(periods=1).fillna(data['te_exp_mean'])
    data['te_lag_1_week'] = data.groupby(by=['country','store','product'])[target_column].shift(periods=7).fillna(data['te_exp_mean'])
    data['te_lag_2_week'] = data.groupby(by=['country','store','product'])[target_column].shift(periods=15).fillna(data['te_exp_mean'])
    data['te_lag_1_month'] = data.groupby(by=['country','store','product'])[target_column].shift(periods=30).fillna(data['te_exp_mean'])
    data['te_lag_3_month'] = data.groupby(by=['country','store','product'])[target_column].shift(periods=91).fillna(data['te_exp_mean'])
    data['te_lag_6_month'] = data.groupby(by=['country','store','product'])[target_column].shift(periods=182).fillna(data['te_exp_mean'])
    data['te_lag_1_year'] = data.groupby(by=['country','store','product'])[target_column].shift(periods=365*1).fillna(data['te_exp_mean'])
    data['te_lag_2_year'] = data.groupby(by=['country','store','product'])[target_column].shift(periods=365*2).fillna(data['te_exp_mean'])
    data['te_lag_3_year'] = data.groupby(by=['country','store','product'])[target_column].shift(periods=365*3).fillna(data['te_exp_mean'])
    data['te_lag_4_year'] = data.groupby(by=['country','store','product'])[target_column].shift(periods=365*4).fillna(data['te_exp_mean'])
    data['te_lag_5_year'] = data.groupby(by=['country','store','product'])[target_column].shift(periods=365*5).fillna(data['te_exp_mean'])

    data['te_rol_2_day_mean'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=2).mean().shift()).fillna(data['te_exp_mean'])
    data['te_rol_1_week_mean'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=2).mean().shift()).fillna(data['te_exp_mean'])
    data['te_rol_2_week_mean'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=15).mean().shift()).fillna(data['te_exp_mean'])
    data['te_rol_1_month_mean'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=30).mean().shift()).fillna(data['te_exp_mean'])
    data['te_rol_3_month_mean'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=91).mean().shift()).fillna(data['te_exp_mean'])
    data['te_rol_6_month_mean'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=182).mean().shift()).fillna(data['te_exp_mean'])
    data['te_rol_1_year_mean'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=365*1).mean().shift()).fillna(data['te_exp_mean'])
    data['te_rol_2_year_mean'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=365*2).mean().shift()).fillna(data['te_exp_mean'])
    data['te_rol_3_year_mean'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=365*3).mean().shift()).fillna(data['te_exp_mean'])
    data['te_rol_4_year_mean'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=365*4).mean().shift()).fillna(data['te_exp_mean'])
    data['te_rol_5_year_mean'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=365*5).mean().shift()).fillna(data['te_exp_mean'])

    data['te_rol_2_day_max'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=2).max().shift()).fillna(data['te_exp_max'])
    data['te_rol_1_week_max'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=2).max().shift()).fillna(data['te_exp_max'])
    data['te_rol_2_week_max'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=15).max().shift()).fillna(data['te_exp_max'])
    data['te_rol_1_month_max'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=30).max().shift()).fillna(data['te_exp_max'])
    data['te_rol_3_month_max'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=91).max().shift()).fillna(data['te_exp_max'])
    data['te_rol_6_month_max'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=182).max().shift()).fillna(data['te_exp_max'])
    data['te_rol_1_year_max'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=365*1).max().shift()).fillna(data['te_exp_max'])
    data['te_rol_2_year_max'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=365*2).max().shift()).fillna(data['te_exp_max'])
    data['te_rol_3_year_max'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=365*3).max().shift()).fillna(data['te_exp_max'])
    data['te_rol_4_year_max'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=365*4).max().shift()).fillna(data['te_exp_max'])
    data['te_rol_5_year_max'] = data.groupby(by=['country','store','product'])[target_column].transform(lambda x: x.rolling(window=365*5).max().shift()).fillna(data['te_exp_max'])

    data = data[~((data['dataset']==True)&(data[target_column].isna()))] # remove the nans in train

    data = data.drop('date', axis=1)
    data = data.drop('dataset', axis=1)
    return data

data = date_features(data)

In [275]:
data["country"] = data["country"].astype('category')
data["store"] = data["store"].astype('category')
data["product"] = data["product"].astype('category')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 319809 entries, 1 to 328679
Data columns (total 47 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   id                   319809 non-null  int64   
 1   country              319809 non-null  category
 2   store                319809 non-null  category
 3   product              319809 non-null  category
 4   date_month           319809 non-null  int32   
 5   date_year            319809 non-null  int32   
 6   date_day             319809 non-null  int32   
 7   date_day_of_week     319809 non-null  int32   
 8   date_day_of_year     319809 non-null  int32   
 9   date_quarter         319809 non-null  int32   
 10  date_month_start     319809 non-null  bool    
 11  date_month_end       319809 non-null  bool    
 12  date_year_start      319809 non-null  bool    
 13  date_year_end        319809 non-null  bool    
 14  date_quarter_start   319809 non-null  bool    
 15  date_

In [276]:
data.head()

,id,country,store,product,date_month,date_year,date_day,date_day_of_week,date_day_of_year,date_quarter,date_month_start,date_month_end,date_year_start,date_year_end,date_quarter_start,date_quarter_end,date_weekend,days_since_start,num_sold,te_exp_mean,te_exp_median,te_exp_std,te_exp_var,te_exp_min,te_exp_max,te_lag_1_day,te_lag_1_week,te_lag_2_week,te_lag_1_month,te_lag_3_month,te_lag_6_month,te_lag_1_year,te_lag_2_year,te_lag_3_year,te_lag_4_year,te_lag_5_year,te_rol_2_day_mean,te_rol_1_week_mean,te_rol_2_week_mean,te_rol_1_month_mean,te_rol_3_month_mean,te_rol_6_month_mean,te_rol_1_year_mean,te_rol_2_year_mean,te_rol_3_year_mean,te_rol_4_year_mean,te_rol_5_year_mean
1,1,Canada,Discount Stickers,Kaggle,1,2010,1,4,1,1,True,False,True,False,True,False,False,0,973.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Canada,Discount Stickers,Kaggle Tiers,1,2010,1,4,1,1,True,False,True,False,True,False,False,0,906.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Canada,Discount Stickers,Kerneler,1,2010,1,4,1,1,True,False,True,False,True,False,False,0,423.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Canada,Discount Stickers,Kerneler Dark Mode,1,2010,1,4,1,1,True,False,True,False,True,False,False,0,491.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,Canada,Stickers for Less,Holographic Goose,1,2010,1,4,1,1,True,False,True,False,True,False,False,0,300.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [277]:
data[(data['country']=='Canada')&(data['store']=='Discount Stickers')&(data['product']=='Kaggle')]

,id,country,store,product,date_month,date_year,date_day,date_day_of_week,date_day_of_year,date_quarter,date_month_start,date_month_end,date_year_start,date_year_end,date_quarter_start,date_quarter_end,date_weekend,days_since_start,num_sold,te_exp_mean,te_exp_median,te_exp_std,te_exp_var,te_exp_min,te_exp_max,te_lag_1_day,te_lag_1_week,te_lag_2_week,te_lag_1_month,te_lag_3_month,te_lag_6_month,te_lag_1_year,te_lag_2_year,te_lag_3_year,te_lag_4_year,te_lag_5_year,te_rol_2_day_mean,te_rol_1_week_mean,te_rol_2_week_mean,te_rol_1_month_mean,te_rol_3_month_mean,te_rol_6_month_mean,te_rol_1_year_mean,te_rol_2_year_mean,te_rol_3_year_mean,te_rol_4_year_mean,te_rol_5_year_mean
1,1,Canada,Discount Stickers,Kaggle,1,2010,1,4,1,1,True,False,True,False,True,False,False,0,973.0,0.000000,0.0,1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
91,91,Canada,Discount Stickers,Kaggle,1,2010,2,5,2,1,False,False,False,False,False,False,True,1,881.0,973.000000,973.0,1.000000,1.000000,973.0,973.0,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000,973.000000
181,181,Canada,Discount Stickers,Kaggle,1,2010,3,6,3,1,False,False,False,False,False,False,True,2,1003.0,927.000000,927.0,65.053824,4232.000000,881.0,973.0,881.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000,927.000000
271,271,Canada,Discount Stickers,Kaggle,1,2010,4,0,4,1,False,False,False,False,False,False,False,3,744.0,952.333333,973.0,63.571482,4041.333333,881.0,1003.0,1003.000000,952.333333,952.333333,952.333333,952.333333,952.333333,952.333333,952.333333,952.333333,952.333333,952.333333,942.000000,942.000000,952.333333,952.333333,952.333333,952.333333,952.333333,952.333333,952.333333,952.333333,952.333333
361,361,Canada,Discount Stickers,Kaggle,1,2010,5,1,5,1,False,False,False,False,False,False,False,4,707.0,900.250000,927.0,116.382630,13544.916667,744.0,1003.0,744.000000,900.250000,900.250000,900.250000,900.250000,900.250000,900.250000,900.250000,900.250000,900.250000,900.250000,873.500000,873.500000,900.250000,900.250000,900.250000,900.250000,900.250000,900.250000,900.250000,900.250000,900.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328231,328231,Canada,Discount Stickers,Kaggle,12,2019,27,4,361,4,False,False,False,False,False,False,False,3647,NaN,715.652718,707.0,96.471406,9306.732243,508.0,1216.0,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,694.000000,776.000000,1078.000000,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718
328321,328321,Canada,Discount Stickers,Kaggle,12,2019,28,5,362,4,False,False,False,False,False,False,True,3648,NaN,715.652718,707.0,96.471406,9306.732243,508.0,1216.0,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,810.000000,798.000000,1002.000000,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718
328411,328411,Canada,Discount Stickers,Kaggle,12,2019,29,6,363,4,False,False,False,False,False,False,True,3649,NaN,715.652718,707.0,96.471406,9306.732243,508.0,1216.0,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,856.000000,717.000000,888.000000,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.652718,715.65271

In [278]:
train_data = data[data[target_column].notna()]
test_data = data[data[target_column].isna()]
test_data = test_data.drop(target_column, axis=1)

test_id = test_data[['id']].copy()
train_data = train_data.drop('id', axis=1)
test_data = test_data.drop('id', axis=1)

In [279]:
X = train_data.drop(target_column, axis=1)
y = (train_data[target_column])

In [280]:
skf = TimeSeriesSplit(n_splits=6)

In [281]:
params = {
          'categorical_feature': 'name:country,store,product',
          'objective': 'regression', 
          'metric':'l1', 
          'n_jobs':-1,
          'seed': 42
}

scores = []
for i, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val)
    
    model = lgb.train(
        params,
        train_data,
        valid_sets=[val_data],
    )

    scores.append(model.best_score['valid_0']['l1'])
print("Mean MAE score:", np.mean(scores))

[LightGBM] [Warning] categorical_feature is set=name:country,store,product, categorical_column=0,1,2 will be ignored. Current value: categorical_feature=name:country,store,product
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003269 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7648
[LightGBM] [Info] Number of data points in the train set: 31611, number of used features: 45
[LightGBM] [Info] Start training from score 713.372244
[LightGBM] [Warning] categorical_feature is set=name:country,store,product, categorical_column=0,1,2 will be ignored. Current value: categorical_feature=name:country,store,product
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004966 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7705
[LightGBM] [Info] Number of data points in the train set: 63219, number of used features: 45
[LightG

In [282]:
train_data = lgb.Dataset(X, label=y)

model = lgb.train(
        params,
        train_data
    )

[LightGBM] [Warning] categorical_feature is set=name:country,store,product, categorical_column=0,1,2 will be ignored. Current value: categorical_feature=name:country,store,product
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015870 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7721
[LightGBM] [Info] Number of data points in the train set: 221259, number of used features: 45
[LightGBM] [Info] Start training from score 752.527382


In [283]:
def getPrediction(estimator, test_id, test_data, target):
    y_pred_submission = estimator.predict(test_data)
    
    test_submission = test_id
    test_submission[target] = y_pred_submission

    return test_submission

In [284]:
test_submission = getPrediction(model, test_id, test_data, target_column)
test_submission.to_csv("lgbm_gbdt_basic.csv", index=False)